In [1]:
import pandas as pd
import numpy as np
import random
import string
import pickle
import os.path
from utils import *

In [2]:
if not os.path.isfile('data/trigrams.pickle'): # run once
    train = open('data/train.txt', 'r') # conll 2003 dataset
    freqs = open('data/freqs.txt', 'r')

    trigrams = trigram_prob(freqs)
    with open('data/trigrams.pickle', 'wb') as handle:
        pickle.dump(tri_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    freqs.seek(0)

    emissions = emission_parameter(freqs)
    with open('data/emissions.pickle', 'wb') as handle:
        pickle.dump(e_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
    freqs.seek(0)

    wordc = word_count(freqs)
    with open('data/wordc.pickle', 'wb') as handle:
        pickle.dump(word_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

    wnet = []
    with open("data/data.noun", "r") as file:
        lines = file.readlines()
        for l in lines:
            if l != "":
                wnet.append(l.split(' ')[4])

    caps = []
    for w in wnet:
        if w != "":
            asc = ord(w[0])
            if asc >= 65 and asc < 91:
                caps.append(w.replace('_', ' '))

    with open('data/wnet_caps.pickle', 'wb') as handle:
        pickle.dump(caps, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
else:
    with open('data/trigrams.pickle', 'rb') as handle:
        trigrams = pickle.load(handle)
    
    with open('data/emissions.pickle', 'rb') as handle:
        emissions = pickle.load(handle)

    with open('data/wordc.pickle', 'rb') as handle:
        wordc = pickle.load(handle)

    with open('data/wnet_caps.pickle', 'rb') as handle:
        caps = pickle.load(handle)

In [3]:
# read dataset
ansar = pd.read_csv("../../Dataset/Ansar1.txt", sep='\t')
ansar = ansar[ansar['Message'].notna()] # drop nan value rows
messages = dict(zip(ansar['MessageID'], ansar['Message']))

In [4]:
def processMessage(message):
    '''
        Takes message, returns english only tokens.
    '''
    printables = filter(lambda x: True if x==' ' else x in string.printable, message)
    eng_text = ""
    for ch in printables:
        eng_text += ch
        
    tokens = nltk.word_tokenize(eng_text)
    for i in range(len(tokens)):
        if tokens[i] is '.':
            tokens.insert(i+1, '')

    if tokens and tokens[-1]:
        tokens.append('')
    
    return tokens

In [ ]:
# predict NER tags for Ansar (takes a long time, don't run, just use messages.rar)
for _id , _msg in messages.items():
    tokens = processMessage(_msg)
    wordtags = into_sentence(tokens, trigrams, emissions, wordc, caps)
    with open('messages/{}.txt'.format(_id), 'w+') as out:
        for word, tag in wordtags:
            if word:
                out.write('{}\t{}\n'.format(word, tag))